# Sistema Tributário: Uma Análise Interseccional

Este notebook tem como objetivo realizar uma análise exploratória dos dados compilados a partir da Pesquisa de Orçamento Familiar(POF).
Os gráficos e análises compiladadas aqui foram utilizados na escrita da NPE XX.

## Passos preliminares

In [ ]:
'''
Importando as bibliotecas de visualização e manipulação de dados. Além disso, também importamos algumas funções da biblioteca
wquantiles, as quais serão utilizadas para encontrar valores de percentis nas distribuições trabalhadas aqui.
'''
import pandas as pd
import numpy as np
from wquantiles import quantile_1D, quantile
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import os
os.chdir('Insira/aqui/seu/diretório/de/trabalho')

In [ ]:
np.seterr(all="ignore")

In [ ]:
# Colocando duas casas decimais como padrão de visualização
pd.set_option('display.float_format', lambda x: '%.10f' % x)

In [ ]:
colors = ["#45ff66", "#eb52ff", "#3366ff","#feff41","#ffab4d","#ff7276","#00b9e3","#8969d3", "#89A577"]
sns.set_palette(sns.color_palette(colors))

Primeiro, importamos a base de dados e criamos uma variável de renda per capita, utilizando as variáveis de Renda Total da unidade de consumo e número de pessoas da mesma.

In [ ]:
pof = pd.read_csv('POFMade2106.csv')

In [ ]:
pof['RENDA_TOTAL_PC'] = pof['RENDA_TOTAL']/pof['N_Pessoas']

A variável V0306 indica se é ou não a pessoa de referência, assim, ao filtrarmos por ela, podemos obter uma POF baseada na pessoa de referência/família

In [ ]:
pof_familia = pof[pof['V0306']==1]

Em seguida, definimos algumas variáveis auxiliares que serão úteis ao longo da exploração dos dados.

In [ ]:
generos = {1: 'H', 2: 'M'}
racas = {'B': [1], 'N': [2,4]}

In [ ]:
impostos = ["IMOVEL_PC", "CONTR_PREVI_PC", "CONT_PREV_EMP_PC", "DEDUCOES_TRABALHO_PC", "DEDUCOES_N_T_PC", "IRPF_PC", "IPVA_PC", "OUT_DIR_PC"]

In [ ]:
transferencias = ['RGPS_T_PC','RPPS_T_PC','AUXI_PUB_T_PC','AUX_MIS_T_PC','PBF_T_PC','BPC_T_PC','BOL_EST_T_PC','SEG_DES_T_PC','OUTRAS_TRANSFERENCIAS_T_PC']

In [ ]:
vetor_show = ["0-50", "50-90", "90-99", "99-100"]

In [ ]:
lista_10_11 = ['0-10','10-20','20-30','30-40','40-50','50-60','60-70','70-80','80-90','90-99','99-100']

Em seguida, criamos 4 novas variáveis dentro de nossa base de dados:
<ul>
    <li> Impostos_PC, a qual reúne todos os impostos(que já estavam per capitalizados) </li>
    <li> Transferencias_PC, a qual reúne todos as transferências(que já estavam per capitalizados) </li>
    <li> Impostos_Incidencia, a qual contém a incidência dos impostos sobre a renda total </li>
    <li> Transferencias_Part, a qual contém a participação das transferências sobre a renda total </li>
</ul>

In [ ]:
pof['Impostos_PC'] = pof[impostos].sum(axis=1)
pof['Transferencias_PC'] = pof[transferencias].sum(axis=1)

In [ ]:
pof['Impostos_Incidencia'] = (pof['Impostos_PC']/pof['RENDA_TOTAL_PC'])*100

In [ ]:
pof['Transferencia_Part'] = (pof['Transferencias_PC']/pof['RENDA_TOTAL_PC'])*100

As células a seguir contém duas funções auxiliares e a criação de uma lista contendo estratos de renda selecionados, os quais serão utilizados ao longo da análise.

In [ ]:
def decis(quantis, pesos):
    lista_decis = []
    vetor_decis = [i/10 for i in range(1,10)]
    for i in vetor_decis:
        lista_decis.append(quantile_1D(quantis,pesos,i))
    return lista_decis
quantis_pc = pof['RENDA_TOTAL_PC'].to_numpy()
pesos = pof['PESO_FINAL'].to_numpy()
decis_pc = [min(quantis_pc)]
decis_pc.extend(decis(quantis_pc, pesos))
decis_pc.append(quantile_1D(quantis_pc,pesos,0.99))
decis_pc.append(max(quantis_pc))

In [ ]:
quantile_1D(quantis_pc,pesos,0.99)

In [ ]:
def mediana_40_1(quantis, pesos):
    lista_decis = []
    vetor_decis = [0.5,0.9,0.99]
    for i in vetor_decis:
        lista_decis.append(quantile_1D(quantis,pesos,i))
    return lista_decis
quantis_pc = pof['RENDA_TOTAL_PC'].to_numpy()
pesos = pof['PESO_FINAL'].to_numpy()
lista_50_40_1 = [min(quantis_pc)]
lista_50_40_1.extend(mediana_40_1(quantis_pc, pesos))
lista_50_40_1.append(max(quantis_pc))

In [ ]:
lista_50_40_1

Nas células a seguir, definimos algumas funções auxiliares para geração de intervalos de confiança.

In [ ]:
def ajustes_esteticos(df):
    df['Deduções de Trabalho'] = df['DEDUCOES_TRABALHO_PC'] + df['DEDUCOES_N_T_PC']
    df.drop('DEDUCOES_TRABALHO_PC', inplace = True, axis = 1)
    df.drop('DEDUCOES_N_T_PC', inplace = True, axis = 1)
    df['Outros Diretos'] = df['CONT_PREV_EMP_PC'] + df['OUT_DIR_PC']
    df.drop('CONT_PREV_EMP_PC', inplace = True, axis = 1)
    df.drop('OUT_DIR_PC', inplace = True, axis = 1)
    df.rename(columns = {'CONTR_PREVI_PC':'Contribuição Previdenciária', 'IRPF_PC':'IRPF', 'IMOVEL_PC': 'Impostos sobre Imóveis', 'IPVA_PC': 'IPVA'}, inplace = True)
    return df

In [ ]:
def ic_95(data):
    ic = 1.96*data['Impostos_Incidencia'].std()/(data.shape[0]**(1/2))
    mean = data['Impostos_Incidencia'].mean()
    return ic

In [ ]:
def ic_95_t(data):
    ic = 1.96*data['Transferencia_Part'].std()/(data.shape[0]**(1/2))
    mean = data['Transferencia_Part'].mean()
    return ic

In [ ]:
def gerar_inter(raca,genero):
    ics=[]
    for i in range(len(lista_50_40_1)):
        if i != 0 and i != len(lista_50_40_1):
            ics.append(ic_95(pof[(pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['V0405'].isin(racas[raca])) & (pof['V0404'] == genero)]))
    return ics

In [ ]:
def gerar_inter_t(raca,genero):
    ics=[]
    for i in range(len(lista_50_40_1)):
        if i != 0 and i != len(lista_50_40_1):
            ics.append(ic_95_t(pof[(pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['V0405'].isin(racas[raca])) & (pof['V0404'] == genero)]))
    return ics

## Composição dos estratos de renda nos diferentes subconjuntos da POF

In [ ]:
tabela_demografica = dict()
for genero in list(generos.keys()):
    for raca in list(racas.values()):
        tabela_demografica[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]] = []
        for i in range(0,len(decis_pc)):
            if i != 0 and i != len(decis_pc):
                tabela_demografica[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]].append((pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL'].sum()/pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1])]['PESO_FINAL'].sum())*100)
tabela_demografica = pd.DataFrame(tabela_demografica, index = lista_10_11)
tabela_demografica = tabela_demografica[['NM','NH','BM','BH']]

In [ ]:
tabela_demografica = tabela_demografica.rename(columns={'NM': 'Mulheres Negras','NH':'Homens Negros','BM': 'Mulheres Brancas','BH':'Homens Brancos'})
tabela_demografica.plot(kind='bar', stacked=True, width=0.8, figsize=(15,8))
plt.legend(bbox_to_anchor=(0.86,-0.25), loc="lower right",ncol=4,fontsize=12)
plt.xlabel("Estratos de renda", fontsize=14)
plt.ylabel("Composição (em %)", fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)

In [ ]:
tabela_demografica_ref = dict()
for genero in list(generos.keys()):
    for raca in list(racas.values()):
        tabela_demografica_ref[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]] = []
        for i in range(0,len(decis_pc)):
            if i != 0 and i != len(decis_pc):
                tabela_demografica_ref[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]].append((pof_familia[(pof_familia['RENDA_TOTAL_PC']<decis_pc[i]) & (pof_familia['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof_familia['V0405'].isin(raca)) & (pof_familia['V0404']==genero)]['PESO_UC'].sum()/pof_familia[(pof_familia['RENDA_TOTAL_PC']<decis_pc[i]) & (pof_familia['RENDA_TOTAL_PC']>=decis_pc[i-1])]['PESO_UC'].sum())*100)
tabela_demografica_ref = pd.DataFrame(tabela_demografica_ref,  index = lista_10_11)
tabela_demografica_ref = tabela_demografica_ref[['NM','NH','BM','BH']]

In [ ]:
tabela_demografica_ref = tabela_demografica_ref.rename(columns={'NM': 'Mulheres Negras','NH':'Homens Negros','BM': 'Mulheres Brancas','BH':'Homens Brancos'})
tabela_demografica_ref.plot(kind='bar', stacked=True, width=0.8, figsize=(15,8))
plt.legend(bbox_to_anchor=(0.86,-0.25), loc="lower right",ncol=4,fontsize=12)
plt.xlabel("Estratos de renda", fontsize=14)
plt.ylabel("Composição (em %)", fontsize=14)
plt.xticks(fontsize=12, rotation=45)
plt.yticks(fontsize=12)

## Impostos

In [ ]:
i=11
raca= [1]
((pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca))]['RENDA_TOTAL_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca))]['PESO_FINAL']).sum())/(pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca))]['PESO_FINAL']).sum()

In [ ]:
i=11
raca= [2,4]
((pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca))]['RENDA_TOTAL_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca))]['PESO_FINAL']).sum())/(pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca))]['PESO_FINAL']).sum()

In [ ]:
impostos_pre_table = dict()
for genero in list(generos.keys()):
    for raca in list(racas.values()):
        impostos_pre_table[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]] = []
        for i in range(0,len(decis_pc)):
            if i != 0 and i != len(decis_pc):
                    impostos_pre_table[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]].append(((pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['Impostos_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum()/
                                                                                                                  (pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['RENDA_TOTAL_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum())*100)

In [ ]:
impostos_tabela = pd.DataFrame(impostos_pre_table, index = lista_10_11)
impostos_tabela = impostos_tabela.rename(columns={'NM': 'Mulheres Negras','NH':'Homens Negros','BM': 'Mulheres Brancas','BH':'Homens Brancos'})
impostos_tabela.plot(kind='bar',width=0.8, figsize=(15,8))
plt.xlabel("Estratos de renda", fontsize=14)
plt.ylabel("Incidência", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.yticks(fontsize=12)
plt.legend(bbox_to_anchor=(0.9,-0.16), loc="lower right",ncol=4,fontsize=12)
#plt.title('Incidência de Impostos Diretos sobre a renda total de grupo demográfico per capita', fontsize = 16)

In [ ]:
impostos_tabela

In [ ]:
impostos_tabela = {'Raça':[],'Imposto':[], 'Incidência':[], 'Decil': [], 'Gênero':[]}
vetor_show = ["0-50", "50-90", "90-99", "99-100"]
for genero in list(generos.keys()):
    for imposto in impostos:
        for raca in list(racas.values()):
            for i in range(0,len(lista_50_40_1)):
                if i != 0 and i != len(lista_50_40_1):
                    impostos_tabela['Incidência'].append(((pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)][imposto]*
                                                                                                                                      pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum()/
                                                                                                                                      (pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['RENDA_TOTAL_PC']*
                                                                                                                                      pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum())*100)
                    impostos_tabela['Raça'].append(list(racas.keys())[list(racas.values()).index(raca)])
                    impostos_tabela['Imposto'].append(imposto)
                    impostos_tabela['Decil'].append(vetor_show[i-1])
                    impostos_tabela['Gênero'].append(generos[genero])

In [ ]:
tabela_impostos = pd.DataFrame(impostos_tabela)

In [ ]:
raca_g = 'B'
genero_g = 'M'
dict_g = {}
for t in impostos:
    dict_g.update({t:list(tabela_impostos[(tabela_impostos['Raça']==raca_g) & (tabela_impostos['Gênero']==genero_g) & (tabela_impostos['Imposto']==t)]['Incidência'])})
df1 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
raca_g = 'B'
genero_g = 'H'
dict_g = {}
for t in impostos:
    dict_g.update({t:list(tabela_impostos[(tabela_impostos['Raça']==raca_g) & (tabela_impostos['Gênero']==genero_g) & (tabela_impostos['Imposto']==t)]['Incidência'])})
df2 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
raca_g = 'N'
genero_g = 'M'
dict_g = {}
for t in impostos:
    dict_g.update({t:list(tabela_impostos[(tabela_impostos['Raça']==raca_g) & (tabela_impostos['Gênero']==genero_g) & (tabela_impostos['Imposto']==t)]['Incidência'])})
df3 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
raca_g = 'N'
genero_g = 'H'
dict_g = {}
for t in impostos:
    dict_g.update({t:list(tabela_impostos[(tabela_impostos['Raça']==raca_g) & (tabela_impostos['Gênero']==genero_g) & (tabela_impostos['Imposto']==t)]['Incidência'])})
df4 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
pd.DataFrame(df1.sum(axis=1)).plot(ax=axes[0,0],kind='bar',width=0.8, figsize=(15,12), title = 'Mulheres Brancas', legend=False, ylim=(0,20), yerr=gerar_inter('B', 2))
pd.DataFrame(df2.sum(axis=1)).plot(ax=axes[0,1],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,20), legend=False, title = 'Homens Brancos', yerr=gerar_inter('B', 1))
pd.DataFrame(df3.sum(axis=1)).plot(ax=axes[1,0],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,20), legend=False, title = 'Mulheres Negras',yerr=gerar_inter('N', 2))
pd.DataFrame(df4.sum(axis=1)).plot(ax=axes[1,1],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,20), legend=False, title = 'Homens Negros', yerr=gerar_inter('N', 1))
plt.savefig('Intervalos2.jpg', dpi = 300)

In [ ]:
df1 = ajustes_esteticos(df1)
df2 = ajustes_esteticos(df2)
df3 = ajustes_esteticos(df3)
df4 = ajustes_esteticos(df4)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
df1.plot(ax=axes[0,0], kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,20), title = 'Mulheres Brancas',legend = False)
df2.plot(ax=axes[0,1],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,20), title = 'Homens Brancos', legend = False)
df3.plot(ax=axes[1,0],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,20), title = 'Mulheres Negras', legend = False)
df4.plot(ax=axes[1,1],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,20), title = 'Homens Negros', legend = False)
h, l = axes[0,0].get_legend_handles_labels()
plt.legend(h,l, bbox_to_anchor=(0.7,-0.35), loc="lower right", ncol=4,fontsize=12)
#fig.suptitle('Incidência de Impostos', fontsize=16)
plt.savefig('Impostos.png')

In [ ]:
emp_dom = dict()
for raca in list(racas.values()):
    emp_dom[list(racas.keys())[list(racas.values()).index(raca)]] = []
    for i in range(0,len(lista_50_40_1)):
        if i != 0 and i != len(lista_50_40_1):
            emp_dom[list(racas.keys())[list(racas.values()).index(raca)]].append(pof_familia[(pof_familia['RENDA_TOTAL_PC']<lista_50_40_1[i]) & (pof_familia['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof_familia['V0405'].isin(raca)) & (pof_familia['ContrEmpDom'].notna())]['PESO_UC'].sum()/pof_familia[(pof_familia['RENDA_TOTAL_PC']<lista_50_40_1[i]) & (pof_familia['RENDA_TOTAL_PC']>=lista_50_40_1[i-1])]['PESO_UC'].sum())


In [ ]:
emp_dom = pd.DataFrame(emp_dom, index = vetor_show)

emp_dom = emp_dom.rename(columns={'N': 'Negros', 'B':'Brancos'})
emp_dom.plot(kind='bar',width=0.8, figsize=(15,8))
plt.xlabel("Estratos de renda", fontsize=14)
plt.ylabel("Representação", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.yticks(fontsize=12)

In [ ]:
cont_prev_emp = dict()
for genero in list(generos.keys()):
    for raca in list(racas.values()):
        cont_prev_emp[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]] = []
        for i in range(0,len(lista_50_40_1)):
            if i != 0 and i != len(decis_pc):
                    cont_prev_emp[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]].append(((pof[(pof['RENDA_TOTAL_PC']<lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['CONT_PREV_EMP_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum()/
                                                                                                                  (pof[(pof['RENDA_TOTAL_PC']<lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['RENDA_TOTAL_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum()))

In [ ]:
cont_prev_emp_tab = pd.DataFrame(cont_prev_emp, index = vetor_show)
cont_prev_emp_tab = cont_prev_emp_tab[['BH','BM','NH','NM']]

cont_prev_emp_tab = cont_prev_emp_tab.rename(columns={'NM': 'Mulheres Negras','NH':'Homens Negros','BM': 'Mulheres Brancas','BH':'Homens Brancos'})
cont_prev_emp_tab.plot(kind='bar',width=0.8, figsize=(15,8))
plt.xlabel("Estratos de renda", fontsize=14)
plt.ylabel("Incidência", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.yticks(fontsize=12)

In [ ]:
transferencias_dem_renda = dict()
for genero in list(generos.keys()):
    for raca in list(racas.values()):
        transferencias_dem_renda[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]] = []
        for i in range(0,len(decis_pc)):
            if i != 0 and i != len(decis_pc):
                    transferencias_dem_renda[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]].append(((pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['Transferencias_PC']*
                        pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum()/
                                                                                                                  (pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['RENDA_TOTAL_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum())*100)

In [ ]:
tabela_trans = pd.DataFrame(transferencias_dem_renda, index = lista_10_11)
tabela_trans = tabela_trans.rename(columns={'NM': 'Mulheres Negras','NH':'Homens Negros','BM': 'Mulheres Brancas','BH':'Homens Brancos'})
tabela_trans.plot(kind='bar',width=0.8, figsize=(15,8))
#plt.title('Participação de Transferências\n na renda total do grupo demográfico', fontsize=18)
plt.xlabel("Estratos de renda", fontsize=14)
plt.ylabel("Participação", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.legend(bbox_to_anchor=(0.9,-0.16), loc="lower right",ncol=4,fontsize=12)
plt.yticks(fontsize=12)

In [ ]:
df1['Grupo Demográfico']=['Mulheres Brancas']*4
df2['Grupo Demográfico']=['Homens Brancos']*4
df3['Grupo Demográfico']=['Mulheres Negras']*4
df4['Grupo Demográfico']=['Homens Negros']*4
df_total = df1.append(df2).append(df3).append(df4)
df_total

In [ ]:
transferencias_tabela = {'Raça':[],'Transferência':[], 'Participação':[], 'Decil': [], 'Gênero':[]}
for genero in list(generos.keys()):
    for transferencia in transferencias:
        for raca in list(racas.values()):
            for i in range(0,len(lista_50_40_1)):
                if i != 0 and i != len(lista_50_40_1):
                    transferencias_tabela['Participação'].append(((pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)][transferencia]*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum()/
                                                                                                                  (pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['RENDA_TOTAL_PC']*
                                                                                                                  pof[(pof['RENDA_TOTAL_PC']<=lista_50_40_1[i]) & (pof['RENDA_TOTAL_PC']>=lista_50_40_1[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']).sum())*100)
                    transferencias_tabela['Raça'].append(list(racas.keys())[list(racas.values()).index(raca)])
                    transferencias_tabela['Transferência'].append(transferencia)
                    transferencias_tabela['Decil'].append(vetor_show[i-1])
                    transferencias_tabela['Gênero'].append(generos[genero])

In [ ]:
transferencias_tabela = pd.DataFrame(transferencias_tabela)

In [ ]:
transferencias_tabela

In [ ]:
raca_g = 'B'
genero_g = 'M'
dict_g = {}
for t in transferencias:
    dict_g.update({t:list(transferencias_tabela[(transferencias_tabela['Raça']==raca_g) & (transferencias_tabela['Gênero']==genero_g) & (transferencias_tabela['Transferência']==t)]['Participação'])})
df1 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
raca_g = 'B'
genero_g = 'H'
dict_g = {}
for t in transferencias:
    dict_g.update({t:list(transferencias_tabela[(transferencias_tabela['Raça']==raca_g) & (transferencias_tabela['Gênero']==genero_g) & (transferencias_tabela['Transferência']==t)]['Participação'])})
df2 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
raca_g = 'N'
genero_g = 'M'
dict_g = {}
for t in transferencias:
    dict_g.update({t:list(transferencias_tabela[(transferencias_tabela['Raça']==raca_g) & (transferencias_tabela['Gênero']==genero_g) & (transferencias_tabela['Transferência']==t)]['Participação'])})
df3 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
raca_g = 'N'
genero_g = 'H'
dict_g = {}
for t in transferencias:
    dict_g.update({t:list(transferencias_tabela[(transferencias_tabela['Raça']==raca_g) & (transferencias_tabela['Gênero']==genero_g) & (transferencias_tabela['Transferência']==t)]['Participação'])})
df4 = pd.DataFrame(dict_g, index = vetor_show)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
pd.DataFrame(df1.sum(axis=1)).plot(ax=axes[0,0],kind='bar',width=0.8, figsize=(15,12), ylim=(0,30), title = 'Mulheres Brancas', legend=False, yerr=gerar_inter_t('B', 2))
pd.DataFrame(df2.sum(axis=1)).plot(ax=axes[0,1],kind='bar',width=0.8, figsize=(15,12), ylim=(0,30),  stacked= True, legend=False, title = 'Homens Brancos', yerr=gerar_inter_t('B', 1))
pd.DataFrame(df3.sum(axis=1)).plot(ax=axes[1,0],kind='bar',width=0.8, figsize=(15,12), ylim=(0,30),  stacked= True, legend=False, title = 'Mulheres Negras',yerr=gerar_inter_t('N', 2))
pd.DataFrame(df4.sum(axis=1)).plot(ax=axes[1,1],kind='bar',width=0.8, figsize=(15,12), ylim=(0,30),  stacked= True, legend=False, title = 'Homens Negros', yerr=gerar_inter_t('N', 1))
plt.savefig('IntervalosTrans.jpg', dpi = 300)

In [ ]:
def ajustes_esteticos_t(df):
    df['Outras Transferências'] = df['OUTRAS_TRANSFERENCIAS_T_PC'] + df['AUXI_PUB_T_PC'] + df['AUX_MIS_T_PC'] + df['BOL_EST_T_PC']
    df.drop('OUTRAS_TRANSFERENCIAS_T_PC', inplace = True, axis = 1)
    df.drop('AUXI_PUB_T_PC', inplace = True, axis = 1)
    df.drop('AUX_MIS_T_PC', inplace = True, axis = 1)
    df.drop('BOL_EST_T_PC', inplace = True, axis = 1)
    df.rename(columns = {'RGPS_T_PC':'RGPS', 'RPPS_T_PC':'RPPS', 'PBF_T_PC': 'Bolsa Família', 'SEG_DES_T_PC': 'Seguro Desemprego', 'BPC_T_PC': 'BPC'}, inplace = True)
    return df
df1 = ajustes_esteticos_t(df1)
df2 = ajustes_esteticos_t(df2)
df3 = ajustes_esteticos_t(df3)
df4 = ajustes_esteticos_t(df4)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2)
df1.plot(ax=axes[0,0], kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,30), title = 'Mulheres Brancas',legend=False)
df2.plot(ax=axes[0,1],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,30), title = 'Homens Brancos',legend=False)
df3.plot(ax=axes[1,0],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,30), title = 'Mulheres Negras',legend=False)
df4.plot(ax=axes[1,1],kind='bar',width=0.8, figsize=(15,12), stacked= True, ylim=(0,30), title = 'Homens Negros',legend=False)
h, l = axes[0,0].get_legend_handles_labels()
plt.legend(h,l, bbox_to_anchor=(0.7,-0.35), loc="lower right", ncol=4,fontsize=12)

In [ ]:
df1['Grupo Demográfico']=['Mulheres Brancas']*4
df2['Grupo Demográfico']=['Homens Brancos']*4
df3['Grupo Demográfico']=['Mulheres Negras']*4
df4['Grupo Demográfico']=['Homens Negros']*4
df_total = df1.append(df2).append(df3).append(df4)
df_total

In [ ]:
iterador = {'Contribuição Previdenciária':['CONTR_PREVI_PC'], 'IRPF':['IRPF_PC'], 'Impostos sobre Imóveis': ['IMOVEL_PC'], 'IPVA': ['IPVA_PC'],'Deduções de Trabalho':['DEDUCOES_TRABALHO_PC','DEDUCOES_N_T_PC'],'Outros Diretos':['CONT_PREV_EMP_PC','OUT_DIR_PC']}

In [ ]:
impostos_pago = dict()
for genero in list(generos.keys()):
    for raca in list(racas.values()):
        impostos_pago[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]] = []
        for chaves in iterador.keys():
            if len(iterador[chaves])>1:
                total_imposto = 0
                total_imposto += (pof[(pof['V0405'].isin(raca))&(pof['V0404']==genero)][iterador[chaves][0]]*pof[(pof['V0405'].isin(raca))&(pof['V0404']==genero)]['PESO_FINAL']).sum()
                total_imposto += (pof[(pof['V0405'].isin(raca))&(pof['V0404']==genero)][iterador[chaves][1]]*pof[(pof['V0405'].isin(raca))&(pof['V0404']==genero)]['PESO_FINAL']).sum()
                impostos_pago[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]].append((total_imposto/((pof[iterador[chaves][0]]+pof[iterador[chaves][1]])*pof['PESO_FINAL']).sum())*100)
            else:
                impostos_pago[list(racas.keys())[list(racas.values()).index(raca)]+generos[genero]].append(((pof[(pof['V0405'].isin(raca))&(pof['V0404']==genero)][iterador[chaves][0]]*pof[(pof['V0405'].isin(raca))&(pof['V0404']==genero)]['PESO_FINAL']).sum()/(pof[iterador[chaves][0]]*pof['PESO_FINAL']).sum())*100)

In [ ]:
impostos_demografico = pd.DataFrame(impostos_pago, index = iterador.keys())
impostos_demografico = impostos_demografico.rename(columns={'NM': 'Mulheres Negras','NH':'Homens Negros','BM': 'Mulheres Brancas','BH':'Homens Brancos'})

In [ ]:
impostos_demografico.to_excel('Imp_Dem.xlsx')

In [ ]:
impostos_demografico

In [ ]:
ocupacoes = list(range(1,8))

In [ ]:
ocupacoes_dict = {'Raça': [], 'Gênero': [], 'Decil': [] ,'Ocupação': [], 'Porcentagem': []}
for genero in list(generos.keys()):
    for ocupacao in ocupacoes:
        for raca in list(racas.values()):
            for i in range(0,len(decis_pc)):
                if i != 0 and i != len(decis_pc):
                    ocupacoes_dict['Porcentagem'].append(((pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero) & (pof['PosiOcup'] == ocupacao)]['PESO_FINAL']*pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero) & (pof['PosiOcup'] == ocupacao)]['RENDA_TOTAL_PC']).sum()/
                    (pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero)]['PESO_FINAL']*pof[(pof['RENDA_TOTAL_PC']<decis_pc[i]) & (pof['RENDA_TOTAL_PC']>=decis_pc[i-1]) & (pof['V0405'].isin(raca)) & (pof['V0404']==genero) & (pof['PosiOcup'] != 0)]['RENDA_TOTAL_PC']).sum())*100)
                    ocupacoes_dict['Gênero'].append(generos[genero])
                    ocupacoes_dict['Raça'].append(list(racas.keys())[list(racas.values()).index(raca)])
                    ocupacoes_dict['Decil'].append(i)
                    ocupacoes_dict['Ocupação'].append(ocupacao)

In [ ]:
ocupacoes_table = pd.DataFrame(ocupacoes_dict)

In [ ]:
raca_g = 'N'
genero_g = 'H'
dict_g = {}
for t in ocupacoes:
    dict_g.update({t:list(ocupacoes_table[(ocupacoes_table['Raça']==raca_g) & (ocupacoes_table['Gênero']==genero_g) & (ocupacoes_table['Ocupação']==t)]['Porcentagem'])})
df1 = pd.DataFrame(dict_g, index = list(range(1,12)))

In [ ]:
df1 = df1.set_axis(['Trabalhador Doméstico', 'Militar', 'Empregado do setor privado', 'Empregado do setor público', 'Empregador','Conta própria','Trabalhador não remunerado em ajuda a membro do domicílio ou parente'], axis=1, inplace=False)

In [ ]:
df1.plot(kind='bar', stacked = True, width=0.8, figsize=(15,8))
plt.xlabel("Decis de renda", fontsize=14)
plt.ylabel("Porcentagem", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.yticks(fontsize=12)

In [ ]:
raca_g = 'B'
genero_g = 'H'
dict_g = {}
for t in ocupacoes:
    dict_g.update({t:list(ocupacoes_table[(ocupacoes_table['Raça']==raca_g) & (ocupacoes_table['Gênero']==genero_g) & (ocupacoes_table['Ocupação']==t)]['Porcentagem'])})
df2 = pd.DataFrame(dict_g, index = list(range(1,12)))

In [ ]:
df2 = df2.set_axis(['Trabalhador Doméstico', 'Militar', 'Empregado do setor privado', 'Empregado do setor público', 'Empregador','Conta própria','Trabalhador não remunerado em ajuda a membro do domicílio ou parente'], axis=1, inplace=False)

In [ ]:
df2.plot(kind='bar', stacked = True, width=0.8, figsize=(15,8))
plt.xlabel("Decis de renda", fontsize=14)
plt.ylabel("Porcentagem", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.yticks(fontsize=12)

In [ ]:
raca_g = 'B'
genero_g = 'M'
dict_g = {}
for t in ocupacoes:
    dict_g.update({t:list(ocupacoes_table[(ocupacoes_table['Raça']==raca_g) & (ocupacoes_table['Gênero']==genero_g) & (ocupacoes_table['Ocupação']==t)]['Porcentagem'])})
df3 = pd.DataFrame(dict_g, index = list(range(1,12)))
df3 = df3.set_axis(['Trabalhador Doméstico', 'Militar', 'Empregado do setor privado', 'Empregado do setor público', 'Empregador','Conta própria','Trabalhador não remunerado em ajuda a membro do domicílio ou parente'], axis=1, inplace=False)
df3.plot(kind='bar', stacked = True, width=0.8, figsize=(15,8))
plt.xlabel("Decis de renda", fontsize=14)
plt.ylabel("Porcentagem", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.yticks(fontsize=12)

In [ ]:
raca_g = 'N'
genero_g = 'M'
dict_g = {}
for t in ocupacoes:
    dict_g.update({t:list(ocupacoes_table[(ocupacoes_table['Raça']==raca_g) & (ocupacoes_table['Gênero']==genero_g) & (ocupacoes_table['Ocupação']==t)]['Porcentagem'])})
df4 = pd.DataFrame(dict_g, index = list(range(1,12)))
df4 = df4.set_axis(['Trabalhador Doméstico', 'Militar', 'Empregado do setor privado', 'Empregado do setor público', 'Empregador','Conta própria','Trabalhador não remunerado em ajuda a membro do domicílio ou parente'], axis=1, inplace=False)
df4.plot(kind='bar', stacked = True, width=0.8, figsize=(15,8))
plt.xlabel("Decis de renda", fontsize=14)
plt.ylabel("Porcentagem", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.yticks(fontsize=12)

In [ ]:
df_new = pd.DataFrame(df1.loc[11,]).transpose()
df_new = df_new.append(pd.DataFrame(df2.loc[11,]).transpose())
df_new = df_new.append(pd.DataFrame(df3.loc[11,]).transpose())
df_new = df_new.append(pd.DataFrame(df4.loc[11,]).transpose())

In [ ]:
df_new.set_axis(['Homens Negros','Homens Brancos','Mulheres Brancas','Mulheres Negras'], axis = 0, inplace = True)

In [ ]:
df_new.to_csv('ocupacoes1porcento.csv')

In [ ]:
df_new.plot(kind='bar', stacked = True, width=0.8, figsize=(15,8))
plt.xlabel("Grupo Demográfico", fontsize=14)
plt.ylabel("Porcentagem", fontsize=14)
plt.xticks(fontsize=12, rotation='horizontal')
plt.title('Composição Empregatícia de Acordo com a Renda \n por grupo demográfico nos 1% mais rico')
plt.legend(bbox_to_anchor=(0.95,-0.3), loc="lower right",ncol=2,fontsize=12)
plt.yticks(fontsize=12)